Step 1: Imports

In [35]:
!pip install duckduckgo-search
!pip install timm
!pip install ddgs
!pip install pyngrok

import os
import time
from ddgs import DDGS
import requests
from PIL import Image
from io import BytesIO

import glob

import shutil
import random

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm

from PIL import Image
import torch

import matplotlib.pyplot as plt
import torch.nn.functional as F

from flask import Flask, request, render_template
import os

from flask import Flask, request, render_template, redirect, url_for
import os
import uuid
from PIL import Image
import torch.nn.functional as F

from flask import Flask, request, render_template, redirect, url_for, send_from_directory
import os
import uuid
from PIL import Image
import torch.nn.functional as F

from pyngrok import ngrok
import threading
import time

Step 2: Preparing the dataset and downloading images

In [36]:
categories = {
    "wood": "wooden toy",
    "fabric": "fabric soft toy",
    "plastic": "plastic toy kids",
    "rubber": "rubber toy kids"
}

dataset_dir = "/content/toy_materials_dataset"
os.makedirs(dataset_dir, exist_ok=True)

def download_images(query, label, max_images=150):
    save_dir = os.path.join(dataset_dir, label)
    os.makedirs(save_dir, exist_ok=True)

    print(f"Downloading {label} images...")
    results = DDGS().images(query, max_results=max_images)

    count = 0
    for img in results:
        try:
            url = img["image"]
            response = requests.get(url, timeout=10)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image.save(os.path.join(save_dir, f"{label}_{count}.jpg"))
            count += 1
            time.sleep(0.5)
        except Exception as e:
            print(f"Error downloading {url}: {e}")
            continue

    print(f"{label}: Downloaded {count} images")

for label, query in categories.items():
    print(f"Downloading: {label}")
    download_images(query, label, max_images=150)
    time.sleep(2)

Downloading: wood
Error downloading http://jvkwoodwork.files.wordpress.com/2012/10/153.jpg: cannot identify image file <_io.BytesIO object at 0x78bcec42f1a0>
Error downloading https://www.crushpixel.com/big-static14/preview4/creative-eco-wooden-toys-baby-1674255.jpg: cannot identify image file <_io.BytesIO object at 0x78bcec7ffc90>
Error downloading https://www.foreststreetdesigns.com/wp-content/uploads/2015/10/0-FIRST-TOYS-HARDWOOD-2.jpg: cannot identify image file <_io.BytesIO object at 0x78bcec7ffc90>
Error downloading http://www.warawoodshed.com.au/wp-content/uploads/2017/03/wooden_toy_car_01.jpg: cannot identify image file <_io.BytesIO object at 0x78bceeeb27a0>
Error downloading http://madebyalan.com/wp-content/uploads/2013/11/IMG_1684.jpg: HTTPSConnectionPool(host='madebyalan.com', port=443): Max retries exceeded with url: /wp-content/uploads/2013/11/IMG_1684.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certifica

Step 3: Removing the corrupted images

In [37]:
def clean_folder(path):
    images = glob.glob(path + "/*")
    for img_path in images:
        try:
            img = Image.open(img_path)
            img.verify()
        except:
            print("Removing:", img_path)
            os.remove(img_path)

for label in categories.keys():
    clean_folder(os.path.join(dataset_dir, label))

Step 4: Creating train/test slip

In [38]:
train_path = "/content/train"
test_path = "/content/test"

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

split_ratio = 0.8

for label in categories.keys():
    src_folder = os.path.join(dataset_dir, label)
    images = os.listdir(src_folder)
    random.shuffle(images)

    split = int(len(images) * split_ratio)
    train_imgs = images[:split]
    test_imgs = images[split:]

    os.makedirs(os.path.join(train_path, label), exist_ok=True)
    os.makedirs(os.path.join(test_path, label), exist_ok=True)

    for img in train_imgs:
        shutil.copy(os.path.join(src_folder, img), os.path.join(train_path, label, img))

    for img in test_imgs:
        shutil.copy(os.path.join(src_folder, img), os.path.join(test_path, label, img))

Step 5: Training EfficientNet-B0 model

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder(train_path, transform=transform)
test_ds = datasets.ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

num_classes = len(categories)

model = timm.create_model("efficientnet_b0", pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 5

Using device: cuda


Step 6: Training it in loop

In [40]:
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")


Epoch 1/5, Loss: 27.2945
Epoch 2/5, Loss: 16.1981
Epoch 3/5, Loss: 10.0587
Epoch 4/5, Loss: 6.4569
Epoch 5/5, Loss: 4.3455


Step 7: Evalutating the Accuarcy

In [41]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.shape[0]

print("Test Accuracy:", correct / total)

Test Accuracy: 0.9645390070921985


Step 8: Prediciting a Image

In [42]:
def predict_and_show(image_path):
    img = Image.open(image_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        confidence, pred_idx = torch.max(probs, 1)

    predicted_class = train_ds.classes[pred_idx.item()]
    confidence_percent = confidence.item() * 100

    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Prediction: {predicted_class}\nConfidence: {confidence_percent:.2f}%")
    plt.show()

    return predicted_class, confidence_percent


Step 9: Creating a web app

In [43]:
get_ipython().system('pip install Flask pyngrok')

Step 9: ii

In [44]:
app = Flask(__name__)

templates_dir = os.path.join(os.getcwd(), 'templates')
os.makedirs(templates_dir, exist_ok=True)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict_image():
    return "Prediction endpoint reached!"

Step 9: iii

In [45]:
html_content = '''
<!DOCTYPE html>
<html>
<head>
    <title>Toy Material Scanner</title>
    <style>
        body { font-family: Times new roman, sans-serif; margin: 100px; background-color: white; color: Blue; }
        .container { background-color: yellow; padding: 70px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); max-width: 600px; margin: auto; text-align: center; }
        h1 { color: blue; }
        input[type="file"] { margin-bottom: 20px; }
        input[type="submit"] { background-color: #007bff; color: white; padding: 10px 15px; border: none; border-radius: 5px; cursor: pointer; font-size: 16px; }
        input[type="submit"]:hover { background-color: #0056b3; }
        .image-display { margin-top: 20px; }
        .image-display img { max-width: 100%; height: auto; border: 1px solid #ddd; border-radius: 4px; }
        .prediction-results { margin-top: 20px; font-size: 1.1em; font-weight: bold; color: #28a745; }
        .error { color: red; margin-top: 10px; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Toy Material Scanner</h1>
        <form action="/predict" method="POST" enctype="multipart/form-data">
            <input type="file" name="image" accept="image/*" required>
            <br>
            <input type="submit" value="Upload and Predict">
        </form>

        {% if filename %}
            <div class="image-display">
                <h2>Uploaded Image:</h2>
                <img src="{{ url_for('static', filename='uploads/' + filename) }}" alt="Uploaded Image">
            </div>
        {% endif %}

        {% if prediction_text %}
            <div class="prediction-results">
                <p>{{ prediction_text }}</p>
            </div>
        {% endif %}

        {% if error %}
            <div class="error">
                <p>{{ error }}</p>
            </div>
        {% endif %}
    </div>
</body>
</html>
'''
with open(os.path.join(templates_dir, 'index.html'), 'w') as f:
    f.write(html_content)

Step 9: iv

In [46]:
app = Flask(__name__)

templates_dir = os.path.join(os.getcwd(), 'templates')
os.makedirs(templates_dir, exist_ok=True)

@app.route('/')
def home():

    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict_image():

    return "Prediction endpoint reached!"

Step 9: v

In [47]:
app = Flask(__name__, static_url_path='/static', static_folder='static')

templates_dir = os.path.join(os.getcwd(), 'templates')
os.makedirs(templates_dir, exist_ok=True)

upload_folder = os.path.join(os.getcwd(), 'static', 'uploads')
os.makedirs(upload_folder, exist_ok=True)

def get_prediction_from_image(image_path):
    img = Image.open(image_path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        probs = F.softmax(output, dim=1)
        confidence, pred_idx = torch.max(probs, 1)

    predicted_class = train_ds.classes[pred_idx.item()]
    confidence_percent = confidence.item() * 100
    return predicted_class, confidence_percent

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict_image():
    if 'image' not in request.files:
        return render_template('index.html', error='No file part')

    file = request.files['image']

    if file.filename == '':
        return render_template('index.html', error='No selected file')

    if file:
        unique_filename = str(uuid.uuid4().hex) + os.path.splitext(file.filename)[1]
        filepath = os.path.join(upload_folder, unique_filename)
        file.save(filepath)

        try:
            predicted_class, confidence_percent = get_prediction_from_image(filepath)
            prediction_text = f"Predicted: {predicted_class} with confidence {confidence_percent:.2f}%"
            return render_template('index.html', filename=unique_filename, prediction_text=prediction_text)
        except Exception as e:
            return render_template('index.html', error=f"Error during prediction: {e}")

    return render_template('index.html', error='An unexpected error occurred.')

Step 9: vi

In [ ]:
NGROK_AUTH_TOKEN = '3608RpnQrJzhYL9udEhM9iPfpvE_TzWtJ4vBzDmhGFvUqbYX'
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

def run_flask_app():
    app.run(port=5000, debug=False, use_reloader=False)

print("Starting Flask app in a background thread...")
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

time.sleep(5)

print("Creating ngrok tunnel...")
public_url = ngrok.connect(5000)
print(f"* Flask app running on: {public_url}")
print("Visit the URL above to access your Smart toy scanner.")